# PX action of equity + etfs

In [ ]:
import requests

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=demo'

r = requests.get(url)

In [51]:
data = r.json()
data

{'Information': 'The **demo** API key is for demo purposes only. Please claim your free API key at (https://www.alphavantage.co/support/#api-key) to explore our full API offerings. It takes fewer than 20 seconds.'}

In [20]:
import pandas as pd
import datetime
ticker = data['Meta Data']['2. Symbol']
del data['Meta Data']
data = data['Time Series (5min)'] #needlesly nested


In [36]:
df = pd.DataFrame.from_dict(data).T
df = df.apply(pd.to_numeric).reset_index()
df = df.rename(columns={'1. open' : 'open',
           '2. high' : 'high',
           '3. low' : 'low',
           '4. close' :'close',
           '5. volume' : 'volume',
           'index' : 'datetime'})
df

,datetime,open,high,low,close,volume
0,2025-10-03 19:55:00,288.3900,288.75,288.3900,288.450,93
1,2025-10-03 19:50:00,288.3813,288.50,288.3800,288.400,56
2,2025-10-03 19:45:00,288.3805,288.74,288.3800,288.740,156
3,2025-10-03 19:40:00,288.3807,288.75,288.3800,288.750,141
4,2025-10-03 19:35:00,288.3800,288.38,288.3800,288.380,212
...,...,...,...,...,...,...
95,2025-10-03 11:55:00,291.1800,291.70,291.0700,291.615,31303
96,2025-10-03 11:50:00,291.2900,291.39,291.0900,291.155,26062
97,2025-10-03 11:45:00,290.8400,291.24,290.6900,291.240,22216
98,2025-10-03 11:40:00,291.0200,291.02,290.4700,290.910,43365


In [37]:
df['datetime'] = pd.to_datetime(df['datetime'])

df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['minute'] = df['datetime'].dt.minute

df['epoch'] = df['datetime'].astype('int64') // 10**9
df = (df
      .set_index('epoch')
      .reindex(columns=[ 'year', 'month', 'day', 'hour', 'minute', 
                'open', 'high', 'low', 'close']))


In [38]:
df

,year,month,day,hour,minute,open,high,low,close
epoch,,,,,,,,,
1759521300,2025,10,3,19,55,288.3900,288.75,288.3900,288.450
1759521000,2025,10,3,19,50,288.3813,288.50,288.3800,288.400
1759520700,2025,10,3,19,45,288.3805,288.74,288.3800,288.740
1759520400,2025,10,3,19,40,288.3807,288.75,288.3800,288.750
1759520100,2025,10,3,19,35,288.3800,288.38,288.3800,288.380
...,...,...,...,...,...,...,...,...,...
1759492500,2025,10,3,11,55,291.1800,291.70,291.0700,291.615
1759492200,2025,10,3,11,50,291.2900,291.39,291.0900,291.155
1759491900,2025,10,3,11,45,290.8400,291.24,290.6900,291.240


In [48]:
def getPxAction(ticker : str, api_key : str, interval : str = '5min') -> pd.DataFrame:
	"""
	Call the api to get a certain ticker as a df,
	with index as epoch timestamps, ohlc, and 
	year, month, day, hour, minute of the price
	----
	Usage:
	>>> getPxAction('IBM', 'demo').head(1)
	epoch       |year   |month  |day    |hour   |minute |open   |high   |low    |close  |
	=====================================================================================
	1759521300  |2025   |10     |03     |19     |55     |288.39 |288.75 |288.39 |288.45 |
	"""

	#the api call
	url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={interval}&apikey={api_key}&extended_hours=false'

	r = requests.get(url)
	data = r.json()[f'Time Series ({interval})']

	df = pd.DataFrame.from_dict(data).T
	df = df.apply(pd.to_numeric).reset_index()
	df = df.rename(columns={'1. open' : 'open',
			'2. high' : 'high',
			'3. low' : 'low',
			'4. close' :'close',
			'5. volume' : 'volume',
			'index' : 'datetime'})
	df['datetime'] = pd.to_datetime(df['datetime'])
	df['year'] = df['datetime'].dt.year
	df['month'] = df['datetime'].dt.month
	df['day'] = df['datetime'].dt.day
	df['hour'] = df['datetime'].dt.hour
	df['minute'] = df['datetime'].dt.minute
	df['epoch'] = df['datetime'].astype('int64') // 10**9
	df = (df
		.set_index('epoch')
		.reindex(columns=[ 'year', 'month', 'day', 'hour', 'minute', 
					'open', 'high', 'low', 'close']))

	return df

In [49]:
getPxAction('IBM', 'demo').head(2)

KeyError: 'Time Series (5min)'

In [ ]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=demo&datatype=csv'


In [56]:
pd.read_csv(url)

,timestamp,open,high,low,close,volume
0,2025-10-03 19:55:00,288.3900,288.75,288.3900,288.450,93
1,2025-10-03 19:50:00,288.3813,288.50,288.3800,288.400,56
2,2025-10-03 19:45:00,288.3805,288.74,288.3800,288.740,156
3,2025-10-03 19:40:00,288.3807,288.75,288.3800,288.750,141
4,2025-10-03 19:35:00,288.3800,288.38,288.3800,288.380,212
...,...,...,...,...,...,...
95,2025-10-03 11:55:00,291.1800,291.70,291.0700,291.615,31303
96,2025-10-03 11:50:00,291.2900,291.39,291.0900,291.155,26062
97,2025-10-03 11:45:00,290.8400,291.24,290.6900,291.240,22216
98,2025-10-03 11:40:00,291.0200,291.02,290.4700,290.910,43365


# ETF Constit

In [71]:
url = 'https://www.alphavantage.co/query?function=ETF_PROFILE&symbol=QQQ&apikey=demo'

r = requests.get(url)

data = r.json()

In [62]:
div_yield = data['dividend_yield']
inc_date = data['inception_date']
data = data['holdings']


In [69]:
df = pd.json_normalize(data)
df = df.rename(columns={'symbol' : 'ticker',
           'description' : 'name'})
df['weight'] = df['weight'].astype('float')

In [73]:
data['net_expense_ratio']

'0.002'